In [1]:
import Nio
import xarray as xr
import math
import glob
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
path = "data/gfsanl_4_20171107_0000_000.grb2"

ds = xr.open_dataset(path, engine="pynio")
df = ds.get(["TMP_P0_L1_GLL0", "UGRD_P0_L100_GLL0", "VGRD_P0_L100_GLL0"]).to_dataframe()

In [3]:
latitudes = df.index.get_level_values("lat_0")
longitudes = df.index.get_level_values("lon_0")

map_function = lambda lon: (lon - 360) if (lon > 180) else lon
remapped_longitudes = longitudes.map(map_function)

df["longitude"] = remapped_longitudes
df["latitude"] = latitudes    

df = df.reset_index(drop=True)
df.columns = ["temperature", "u_component", "v_component", "longitude", "latitude"]

In [4]:
# Small data: 1/16 times original data
df_small = df.iloc[0:503595].copy()
df_small

,temperature,u_component,v_component,longitude,latitude
0,247.178085,-38.843361,4.000019,0.0,90.0
1,247.178085,-35.828548,1.100024,0.0,90.0
2,247.178085,-36.096478,0.199982,0.0,90.0
3,247.178085,-34.036407,-0.800006,0.0,90.0
4,247.178085,-31.010010,-0.700012,0.0,90.0
...,...,...,...,...,...
503590,253.978073,0.940039,4.868413,-158.0,79.0
503591,253.978073,-0.510283,5.938098,-158.0,79.0
503592,253.978073,-2.153777,6.939585,-158.0,79.0
503593,253.978073,-3.987563,8.009817,-158.0,79.0


In [5]:
8057520/16

503595.0

In [6]:
# Big data: 64 times original data
from tqdm import tqdm
for i in tqdm(range(6)):
    df = pd.concat([df,df])
df = df.reset_index(drop=True)

100%|██████████| 6/6 [00:06<00:00,  1.07s/it]


In [7]:
df

,temperature,u_component,v_component,longitude,latitude
0,247.178085,-38.843361,4.000019,0.0,90.0
1,247.178085,-35.828548,1.100024,0.0,90.0
2,247.178085,-36.096478,0.199982,0.0,90.0
3,247.178085,-34.036407,-0.800006,0.0,90.0
4,247.178085,-31.010010,-0.700012,0.0,90.0
...,...,...,...,...,...
515681275,243.478073,1.780039,-6.711587,-0.5,-90.0
515681276,243.478073,1.779717,-6.711902,-0.5,-90.0
515681277,243.478073,1.776223,-6.710415,-0.5,-90.0
515681278,243.478073,1.772436,-6.710183,-0.5,-90.0


# Calculate Wind Speed and Direction

In [8]:
def find_wind_speed_and_direction(u, v):
    if u == 0 and v == 0:
        return 0, 0
    
    wind_speed = math.sqrt(u**2 + v**2)
    wind_speed *= 18/5
    
    wind_direction = math.atan2(u/wind_speed, v/wind_speed) + math.pi
    wind_direction *= 180/math.pi
    
    return wind_speed, wind_direction

## Simple Loop (CPU)

In [9]:
%%time
lst_wind_speed = []
lst_wind_direction = []

u_component = df_small["u_component"]
v_component = df_small["v_component"]

arr_size = u_component.shape[0]

for i in range(arr_size):
    u = u_component[i]
    v = v_component[i]
    wind_speed, wind_direction = find_wind_speed_and_direction(u,v)
    lst_wind_speed.append(wind_speed)
    lst_wind_direction.append(wind_direction)
    
df_small["wind_speed"] = lst_wind_speed
df_small["wind_direction"] = lst_wind_direction

CPU times: user 4.23 s, sys: 6.62 ms, total: 4.24 s
Wall time: 4.24 s


In [10]:
df_small[["wind_speed", "wind_direction"]]

,wind_speed,wind_direction
0,140.575590,95.879490
1,129.043552,91.758569
2,129.949313,90.317427
3,122.564909,88.653544
4,111.664475,88.706839
...,...,...
503590,17.850018,190.928713
503591,21.455938,175.088424
503592,26.158055,162.757692
503593,32.211015,153.534294


## Pandas Apply (CPU)

In [11]:
%%time
df_small[["wind_speed", "wind_direction"]] = df_small[["u_component", "v_component"]].apply(lambda x: find_wind_speed_and_direction(*x), axis=1, result_type="expand")

CPU times: user 4.97 s, sys: 140 ms, total: 5.11 s
Wall time: 5.11 s


In [12]:
df_small[["wind_speed", "wind_direction"]]

,wind_speed,wind_direction
0,140.575590,95.879490
1,129.043552,91.758569
2,129.949313,90.317427
3,122.564909,88.653544
4,111.664475,88.706839
...,...,...
503590,17.850018,190.928713
503591,21.455938,175.088424
503592,26.158055,162.757692
503593,32.211015,153.534294


## Numpy (CPU)

In [13]:
import numpy as np

In [14]:
u_component_np_small = np.array(df_small["u_component"])
v_component_np_small = np.array(df_small["v_component"])

In [15]:
u_component_np = np.array(df["u_component"])
v_component_np = np.array(df["v_component"])

In [16]:
def find_wind_speed_and_direction_np(u, v):
    wind_speed = np.sqrt(u**2 + v**2)
    wind_speed *= 18/5
    
    wind_direction = np.arctan2(u/wind_speed, v/wind_speed) + math.pi
    wind_direction *= 180/math.pi
    
    wind_direction[wind_direction == float("Inf")] = 0
    wind_direction[wind_direction == float("-Inf")] = 0
    
    return wind_speed, wind_direction

In [17]:
%%time
# small data
wind_speed, wind_direction = find_wind_speed_and_direction_np(u_component_np_small,v_component_np_small)

CPU times: user 2.33 ms, sys: 0 ns, total: 2.33 ms
Wall time: 2.16 ms


In [18]:
%%time
# big data
wind_speed, wind_direction = find_wind_speed_and_direction_np(u_component_np,v_component_np)

CPU times: user 2.34 s, sys: 1.72 s, total: 4.06 s
Wall time: 4.06 s


In [19]:
df["wind_speed"] = wind_speed
df["wind_direction"] = wind_direction

df[["wind_speed", "wind_direction"]]

,wind_speed,wind_direction
0,140.575592,95.879494
1,129.043549,91.758575
2,129.949310,90.317429
3,122.564903,88.653549
4,111.664474,88.706841
...,...,...
515681275,24.997051,345.146088
515681276,24.997850,345.149292
515681277,24.989454,345.174042
515681278,24.985163,345.203766


## CuPy (GPU)

In [20]:
import cupy as cp

In [21]:
u_component_cupy = cp.array(df["u_component"])
v_component_cupy = cp.array(df["v_component"])

In [22]:
def find_wind_speed_and_direction_cupy(u, v):
    wind_speed = cp.sqrt(u**2 + v**2)
    wind_speed *= 18/5
    
    wind_direction = cp.arctan2(u/wind_speed, v/wind_speed) + cp.pi
    wind_direction *= 180/cp.pi
    
    wind_direction[wind_direction == float("Inf")] = 0
    wind_direction[wind_direction == float("-Inf")] = 0
    
    return wind_speed, wind_direction

In [23]:
%%time
wind_speed, wind_direction = find_wind_speed_and_direction_cupy(u_component_cupy,v_component_cupy)

CPU times: user 57.7 ms, sys: 9 ms, total: 66.7 ms
Wall time: 75 ms


In [24]:
df["wind_speed"] = wind_speed.get()
df["wind_direction"] = wind_direction.get()

df[["wind_speed", "wind_direction"]]

,wind_speed,wind_direction
0,140.575592,95.879494
1,129.043549,91.758575
2,129.949310,90.317429
3,122.564903,88.653549
4,111.664474,88.706841
...,...,...
515681275,24.997051,345.146088
515681276,24.997850,345.149292
515681277,24.989454,345.174042
515681278,24.985163,345.203766


In [25]:
del u_component_cupy
del v_component_cupy

## Pytorch (GPU)

In [26]:
import numpy as np
import torch

In [27]:
u_component = np.array(df["u_component"])
v_component = np.array(df["v_component"])

u_component_pytorch = torch.tensor(u_component, dtype=torch.float64).to('cuda:0')
v_component_pytorch = torch.tensor(v_component, dtype=torch.float64).to('cuda:0')

In [28]:
def find_wind_speed_and_direction_pytorch(u, v):
    wind_speed = torch.sqrt(u**2 + v**2)
    wind_speed *= 18/5
    
    wind_direction = torch.atan2(u/wind_speed, v/wind_speed) + math.pi
    wind_direction *= 180/math.pi
    
    wind_direction[wind_direction == float("Inf")] = 0
    wind_direction[wind_direction == float("-Inf")] = 0
    
    return wind_speed, wind_direction

In [29]:
%%time
wind_speed, wind_direction = find_wind_speed_and_direction_pytorch(u_component_pytorch,v_component_pytorch)

CPU times: user 56.7 ms, sys: 48.8 ms, total: 106 ms
Wall time: 105 ms


In [30]:
df["wind_speed"] = wind_speed.to('cpu').numpy()
df["wind_direction"] = wind_direction.to('cpu').numpy()

df[["wind_speed", "wind_direction"]]

,wind_speed,wind_direction
0,140.575590,95.879490
1,129.043552,91.758569
2,129.949313,90.317427
3,122.564909,88.653544
4,111.664475,88.706839
...,...,...
515681275,24.997053,345.146060
515681276,24.997850,345.149296
515681277,24.989456,345.174025
515681278,24.985164,345.203754
